<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://xgboost.readthedocs.io/en/latest/index.html" target="_blank">XGBoost</a></a></span><ul class="toc-item"><li><span><a href="#Baseline-Model" data-toc-modified-id="Baseline-Model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Baseline Model</a></span><ul class="toc-item"><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Prepare Data</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Average-Treatment-Effect" data-toc-modified-id="Average-Treatment-Effect-1.1.3.1"><span class="toc-item-num">1.1.3.1&nbsp;&nbsp;</span>Average Treatment Effect</a></span></li></ul></li></ul></li><li><span><a href="#New-Model" data-toc-modified-id="New-Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>New Model</a></span><ul class="toc-item"><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Prepare Data</a></span><ul class="toc-item"><li><span><a href="#Categorical-Features" data-toc-modified-id="Categorical-Features-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>Categorical Features</a></span></li><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-1.2.1.2"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>Pipeline</a></span></li></ul></li><li><span><a href="#Train" data-toc-modified-id="Train-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></li><li><span><a href="#Features-Importance" data-toc-modified-id="Features-Importance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Features Importance</a></span><ul class="toc-item"><li><span><a href="#SHAP" data-toc-modified-id="SHAP-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://github.com/slundberg/shap" target="_blank">SHAP</a></a></span></li><li><span><a href="#XGBoost-features-importance" data-toc-modified-id="XGBoost-features-importance-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span><a href="https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster.get_score" target="_blank">XGBoost features importance</a></a></span></li></ul></li><li><span><a href="#Practical-Lessons-From-Facebook" data-toc-modified-id="Practical-Lessons-From-Facebook-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://quinonero.net/Publications/predicting-clicks-facebook.pdf" target="_blank">Practical Lessons From Facebook</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Prepare Data</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Make-a-Submission" data-toc-modified-id="Make-a-Submission-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Make a Submission</a></span></li></ul></li></ul></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

# [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html)


Утановим xgboost

In [1]:
!pip3.5 install xgboost

In [2]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

sys.path.append('./utils')

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar
--py-files sparkxgb.zip pyspark-shell
""".replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne
from processing import split_by_col

from sparkxgb.xgboost import *

Обратите внимание на слудующие строки:

* ```python
sys.path.append('./utils')
...
from metrics import rocauc, logloss, ne
from processing import split_by_col
```

В папке `utils` находится два файла (`metrics.py`, `processing.py`), которые содержат функции, которые нужно было реализовать в рамках предыдущей практики.


* ```python
os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar
--py-files sparkxgb.zip pyspark-shell
"""
...
from sparkxgb.xgboost import *
```

Для того чтобы в рамках инфраструктуры Spark можно было использовать XGBoost, мы воспользуемся библиотекой [XGBoost4J](https://xgboost.readthedocs.io/en/latest/jvm/xgboost4j_spark_tutorial.html).

В ходе выполнения занятий может быть полезно ознакомиться с исходным кодом обертки для питона, который находится в архиве `sparkxgb.zip`

In [3]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [4]:
all_metrics = {}

## Baseline Model
### Prepare Data

In [5]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [6]:
df = df.sample(False, 0.5)

In [7]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [8]:
df = df.fillna(0, subset=num_columns)

Воспользуемся пайплайном из предыдущей практики

In [9]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [10]:
pipeline_model.stages

[StringIndexer_a0c62ded8c44,
 StringIndexer_99ed705758c4,
 OneHotEncoderEstimator_2fee80beba37,
 VectorAssembler_c041e3a5389b]

In [11]:
len(pipeline_model.stages[0].labels), len(pipeline_model.stages[1].labels)

(1403, 552)

Итоговая размерность пространства фичей

In [12]:
dim = len(num_columns) + len(pipeline_model.stages[0].labels) + len(pipeline_model.stages[1].labels)
dim

1968

In [13]:
df_transformed = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df_transformed.count()

1832068

In [14]:
train_df, val_df, test_df = split_by_col(df_transformed, 'id', [0.8, 0.1, 0.1])

train_df.take(10)

[Row(label=1, features=SparseVector(1968, {1: -1.0, 4: 1465.0, 6: 17.0, 8: 4.0, 10: 4.0, 25: 1.0, 1416: 1.0}), id=12),
 Row(label=1, features=SparseVector(1968, {0: 88.0, 1: 319.0, 3: 4.0, 4: 5.0, 5: 4.0, 6: 89.0, 7: 40.0, 8: 88.0, 9: 3.0, 10: 4.0, 11: 12.0, 12: 4.0, 13: 1.0, 1427: 1.0}), id=41),
 Row(label=0, features=SparseVector(1968, {1: 53.0, 3: 10.0, 4: 6550.0, 5: 98.0, 6: 34.0, 7: 11.0, 8: 349.0, 10: 9.0, 12: 10.0, 13: 1.0, 1417: 1.0}), id=85),
 Row(label=0, features=SparseVector(1968, {1: 5.0, 2: 30.0, 3: 4.0, 7: 5.0, 8: 4.0, 12: 5.0, 14: 1.0, 1417: 1.0}), id=108),
 Row(label=0, features=SparseVector(1968, {1: 179.0, 2: 4.0, 3: 4.0, 4: 17232.0, 7: 18.0, 8: 7.0, 12: 4.0, 13: 1.0, 1423: 1.0}), id=151),
 Row(label=0, features=SparseVector(1968, {1: 18.0, 2: 15.0, 3: 9.0, 4: 4494.0, 7: 9.0, 8: 8.0, 12: 9.0, 13: 1.0, 1428: 1.0}), id=175),
 Row(label=0, features=SparseVector(1968, {1: 76.0, 2: 3.0, 4: 5029.0, 7: 16.0, 13: 1.0, 1423: 1.0}), id=206),
 Row(label=1, features=SparseVector

### Train

In [15]:
estimator = XGBoostEstimator(
    featuresCol="features", 
    labelCol="label", 
    predictionCol="prediction",
    colsample_bytree=0.9,
    eta=0.15,
    gamma=0.9,
    max_depth=8,
    min_child_weight=50.0,
    subsample=0.9,
    objective='binary:logistic',
    eval_metric='logloss', 
    silent=0,
    num_round=20,
    nthread=1,
    nworkers=1
)

In [16]:
model = estimator.fit(train_df)

Сохраняем [booster](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster) обученной модели

In [17]:
model._call_java("booster").saveModel(os.path.join(DATA_PATH, 'xgb.model'))

### Evaluation

In [18]:
baseline_metrics = {}

#### Average Treatment Effect

Пусть даны две экспериментальные группы treatment ($T$) и control ($C$), где

* `treatment` - группа с изменением (например, новая модель)
* `control` - группа без изменений

Рассмотрим метрику $X$, значение которой мы расчитали для наших групп ($X_T, X_C$).

Тогда под ATE будем иметь в виду
$$ \Delta\% = \frac{X_T - X_C}{X_C} \cdot 100 $$

In [19]:
import pandas as pd

def get_ate(groups, control_name) -> pd.DataFrame:
    """Get Average Treatment Effect
    groups - dictionary where keys - names of models, values - dicts of pairs <metric_name>, <metric_value>
    control_name - name of baseline model
    
    return pd.DataFrame (rows corresponds to metrics, cols corresponds to models and ATE with respect to control)
    """
    ate = {}
    for model in groups.keys():
        ate[model] = {}
        for metric_name in groups[model].keys():
            ate[model][metric_name] = groups[model][metric_name] / groups[control_name][metric_name] * 100
    return pd.DataFrame(ate)

In [20]:
rocauc(model, val_df, probabilities_col='probabilities')

0.7282804796291293

In [21]:
baseline_metrics['ROC AUC'] = rocauc(model, test_df, probabilities_col='probabilities')
baseline_metrics['ROC AUC']

0.7273897037633462

In [22]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import math
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

def logloss(model, df, probabilities_col):
    pred = model.transform(df)

    firstelementUdf = udf(lambda v:float(v[1]), FloatType())
    df_logloss = pred \
        .withColumn(
            'logloss'
            , -F.col('label') * F.log(firstelementUdf(probabilities_col)) - (1. - F.col('label')) * F.log(1. - (firstelementUdf(probabilities_col)))
        )

    res_value = float(df_logloss \
        .agg(F.mean('logloss')) \
        .head()[0])


    return res_value


def ne(model, df, probabilities_col):
    ll = logloss(model, df, probabilities_col)
    p = df.agg(F.avg(F.col('label'))).head()[0]
    const_entropy = -p * math.log(p) - (1. - p) * math.log(1. - p)
    return ll / const_entropy

In [23]:
baseline_metrics['LOG LOSS'] = logloss(model, test_df, 'probabilities')
baseline_metrics['NE'] = ne(model, test_df, 'probabilities')

In [24]:

all_metrics['xgb_baseline'] = baseline_metrics

Сравним результаты с логрег моделью из предыдущей практики.

1. Загрузить обученную `LogReg` модель
2. Посчитать метрики на `test_df`

In [25]:
from pyspark.ml.classification import LogisticRegressionModel

lr_model = LogisticRegressionModel.load(os.path.join(DATA_PATH, 'lr_model'))

baseline_metrics_lr = {}
baseline_metrics_lr['ROC AUC'] = rocauc(lr_model, test_df, probabilities_col='probability')
baseline_metrics_lr['LOG LOSS'] = logloss(lr_model, test_df, 'probability')
baseline_metrics_lr['NE'] = ne(lr_model, test_df, 'probability')

all_metrics['lr_model'] = baseline_metrics_lr

Построить таблицу ATE используя метод `get_ate`

In [26]:
ate = get_ate(all_metrics, 'lr_model')
ate

,lr_model,xgb_baseline
LOG LOSS,100.0,93.688814
NE,100.0,93.688814
ROC AUC,100.0,104.248229


## New Model
### Prepare Data

In [27]:
# df = sqlContext.read.format("com.databricks.spark.csv") \
#     .option("delimiter", ",") \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .load('file:///' + TRAIN_PATH)

# df = df.sample(False, 0.5)
# df = df.fillna(0, subset=num_columns)

#### Categorical Features

See [Doc](https://spark.apache.org/docs/latest/ml-pipeline.html) for additional details on Transformers and Encoders.

Implement classes below

In [28]:

# Fitted Model
class MeanTargetEncoderModel(pyspark.ml.Model, 
                             pyspark.ml.util.DefaultParamsReadable, 
                             pyspark.ml.util.DefaultParamsWritable):
    
    def __init__(self, inputCol, featuresCol, encoded_feature_df):
        super(MeanTargetEncoderModel, self).__init__()
        self.inputCol = inputCol
        self.featuresCol = featuresCol
        self.encoded_feature_df = encoded_feature_df


    def transform(self, df):
        return df \
            .join(self.encoded_feature_df, [self.inputCol], how='left_outer') \
            .fillna(0, subset=[self.featuresCol])


# Estimator
class MeanTargetEncoder(pyspark.ml.Estimator):
    
    def __init__(self, inputCol, targetCol, featuresCol):
#         super(MeanTargetEncoder, self).__init__()
        self.inputCol = inputCol
        self.targetCol = targetCol
        self.featuresCol = featuresCol


    def fit(self, df):
        encoded_feature_df = df \
            .groupby(self.inputCol) \
            .agg(F.mean(self.targetCol).alias(self.featuresCol))
        return MeanTargetEncoderModel(self.inputCol, self.featuresCol, encoded_feature_df)


#### Pipeline

In [29]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler


cat_enc_columns = [cat_col + '_enc' for cat_col in cat_columns]

mean_target_encoders = [MeanTargetEncoder(cat_col, '_c0', cat_enc_col) 
                        for cat_col, cat_enc_col in zip(cat_columns, cat_enc_columns)]

assembler = VectorAssembler(inputCols=num_columns + cat_enc_columns, outputCol="features").setHandleInvalid("keep")

pipeline = Pipeline(stages=mean_target_encoders + [assembler])

In [30]:
pipeline_model = pipeline.fit(df)
# pipeline_model.save(os.path.join(DATA_PATH, 'pipeline_model_2'))

In [31]:
# from pyspark.ml import PipelineModel

# pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model_2'))

In [32]:
df_transformed = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df_transformed.count()

1832068

In [33]:
df_transformed.take(10)

[Row(label=0, features=DenseVector([0.0, 2.0, 0.0, 0.0, 3683.0, 58.0, 4.0, 19.0, 105.0, 0.0, 2.0, 0.0, 0.0, 0.2555, 0.2239]), id=120259130921),
 Row(label=1, features=DenseVector([0.0, 1.0, 3.0, 2.0, 1344.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 2.0, 0.2555, 0.2239]), id=420907297433),
 Row(label=0, features=DenseVector([1.0, 1.0, 5.0, 41.0, 27.0, 43.0, 7.0, 47.0, 197.0, 1.0, 4.0, 0.0, 40.0, 0.2555, 0.2239]), id=446676645620),
 Row(label=0, features=DenseVector([0.0, 0.0, 21.0, 9.0, 4017.0, 236.0, 1.0, 23.0, 82.0, 0.0, 1.0, 1.0, 12.0, 0.2556, 0.2239]), id=17180309479),
 Row(label=1, features=DenseVector([0.0, -1.0, 5.0, 5.0, 1524.0, 350.0, 1.0, 44.0, 143.0, 0.0, 1.0, 0.0, 5.0, 0.2556, 0.2239]), id=34360175192),
 Row(label=1, features=SparseVector(15, {1: 3.0, 4: 40412.0, 5: 203.0, 8: 19.0, 13: 0.2556, 14: 0.2239}), id=42949981847),
 Row(label=0, features=DenseVector([0.0, 0.0, 5.0, 5.0, 16364.0, 71.0, 3.0, 4.0, 46.0, 0.0, 1.0, 0.0, 5.0, 0.2556, 0.2239]), id=85899349708),
 Row(label=0, f

### Train

Train XGBoost on the new set of features

In [34]:
train_df, val_df, test_df = split_by_col(df_transformed, 'id', [0.8, 0.1, 0.1])

estimator = XGBoostEstimator(
    featuresCol="features", 
    labelCol="label", 
    predictionCol="prediction",
    colsample_bytree=0.9,
    eta=0.15,
    gamma=0.9,
    max_depth=8,
    min_child_weight=50.0,
    subsample=0.9,
    objective='binary:logistic',
    eval_metric='logloss', 
    silent=0,
    num_round=20,
    nthread=1,
    nworkers=1
)

model = estimator.fit(train_df)

### Evaluation

Сравнить результаты новой модели с `xgb_baseline` и `log_reg` с помощью функции `get_ate`

In [35]:
baseline_metrics_xgb_mte = {}
baseline_metrics_xgb_mte['ROC AUC'] = rocauc(model, test_df, probabilities_col='probabilities')
baseline_metrics_xgb_mte['LOG LOSS'] = logloss(model, test_df, 'probabilities')
baseline_metrics_xgb_mte['NE'] = ne(model, test_df, 'probabilities')

all_metrics['xgb_mte'] = baseline_metrics_xgb_mte
all_metrics['xgb_mte']

{'LOG LOSS': 0.5062122600853696,
 'NE': 0.8790873945611208,
 'ROC AUC': 0.7353449617412758}

In [36]:
ate = get_ate(all_metrics, 'lr_model')
ate

,lr_model,xgb_baseline,xgb_mte
LOG LOSS,100.0,93.688814,92.834176
NE,100.0,93.688814,92.834176
ROC AUC,100.0,104.248229,105.388363


In [37]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

df_test = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

df_test.printSchema()

df_test = df_test.fillna(0, subset=num_columns)

root
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = true)
 |-- _c28: string (

In [38]:
df_test_transformed = pipeline_model \
    .transform(df_test) \
    .select('features', 'id')

firstelementUdf = udf(lambda v: float(v[1]), FloatType())

pred_test_df = model \
        .transform(df_test_transformed) \
        .select('id', 'probabilities') \
        .withColumn('proba', firstelementUdf('probabilities')) \
        .drop('probabilities')

pred_test_df.show(5)

+------------+----------+
|          id|     proba|
+------------+----------+
|592705839438| 0.3209713|
|670015250683|0.28500983|
|704375045413|0.17341448|
|566936227781|0.18609166|
|618475712763|0.34454244|
+------------+----------+
only showing top 5 rows



In [39]:
TEST_PREDICTIONS_PATH = 'test_pred_xgb_mte'

pred_test_df \
    .coalesce(1) \
    .write \
    .option("header", "true") \
    .mode('overwrite') \
    .csv('/workspace/data/' + TEST_PREDICTIONS_PATH)

Py4JJavaError: An error occurred while calling o2253.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

# Features Importance

## [SHAP](https://github.com/slundberg/shap)

Install SHAP

In [ ]:
!pip3.5 install shap

In [ ]:
import shap
import xgboost as xgb

# load JS visualization code to notebook
shap.initjs()

bst = xgb.Booster()
bst.load_model(os.path.join(DATA_PATH, 'xgb.model'))

Имея `booster` модели можно, например, посмотреть на то какие деревья получились в итоге.

In [ ]:
print(bst.get_dump()[0])

In [ ]:
sample_df = df.sample(False, 0.05)
sample_df.count()

In [ ]:
from scipy.sparse import csr_matrix


def df_to_csr(df, dim):
    data = []
    row_ind = []
    col_ind = []
    
    sparse_vecs = df.rdd.map(lambda row: row.features).collect()
    for i, vec in enumerate(sparse_vecs):
        for idx, val in zip(vec.indices, vec.values):
            data.append(val)
            row_ind.append(i)
            col_ind.append(idx)
        
    return csr_matrix((data, (row_ind, col_ind)), shape=(len(sparse_vecs), dim))

In [ ]:
%%time
X = df_to_csr(sample_df, dim)
dtest = xgb.DMatrix(X)

In [ ]:
X_arr = X.toarray()

In [ ]:
%%time
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(dtest, tree_limit=20)

In [ ]:
shap.summary_plot(shap_values, X_arr, max_display=20, alpha=0.3)

## [XGBoost features importance](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster.get_score)

In [ ]:
def get_feature_score(booster, importance):
    gains_xgb = booster.get_score(importance_type=importance)
    gains = {}
    for f, g in gains_xgb.items():
        gains[f] = g
    sorted_gains = sorted(list(gains.items()), key=lambda x: -x[1])
    return sorted_gains


features_scores = get_feature_score(bst, 'gain')[:20]

In [ ]:
import pandas as pd


f_names, f_scores = zip(*features_scores)
features_scores_pdf = pd.DataFrame({'feature': f_names, 'gain': f_scores})

In [ ]:
import seaborn as sns


_, ax = plt.subplots(figsize=(8,8))
ax = sns.barplot(x='gain', y='feature', data=features_scores_pdf)
plt.show()

# [Practical Lessons From Facebook](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

## Prepare Data

* Реализуйте модель из статьи (LogReg поверх XGBoost)

* Попробуйте реализовать Negatives Subsampling + Re-calibration описанный в статье (доп. баллы)

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

## Train

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

## Evaluation

In [ ]:
def calibration(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass

Сравните новую модель со всеми предыдущими с помощью `get_ate`. При сравнении использовать еще и метрику calibration

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

### Make a Submission

Если в результате работы получилась модель, которая лучше чем ЛогРег из предыдущей практики, то точно нужно сделать submission.